In [1]:
import pandas as pd
import xlwings as xw
import numpy as np

In [2]:
trip_2018_06_df = pd.read_csv('trip_data/summer_months/201806-citibike-tripdata.csv')
print(f'done with trip_2018_06_df')
trip_2018_07_df = pd.read_csv('trip_data/summer_months/201807-citibike-tripdata.csv')
print(f'done with trip_2018_07_df')
trip_2018_08_df = pd.read_csv('trip_data/summer_months/201808-citibike-tripdata.csv')
print(f'done with trip_2018_08_df')

done with trip_2018_06_df
done with trip_2018_07_df
done with trip_2018_08_df


How many trips have been recorded total during the chosen period?

In [3]:
summer_2018_df = pd.concat([trip_2018_06_df,trip_2018_07_df,trip_2018_08_df])
summer_2018_df = summer_2018_df.sort_values(by = 'birth year', ascending = True)
summer_2018_df = summer_2018_df[(summer_2018_df['birth year'] >= 1940)]


summer_2018_df['Date'] = pd.to_datetime(summer_2018_df['starttime'])
summer_2018_df['Month'] = summer_2018_df['Date'].dt.month
summer_2018_df['Year'] = summer_2018_df['Date'].dt.year
summer_2018_df['Hour'] = summer_2018_df['Date'].dt.hour


summer_2018_df = summer_2018_df.sort_values(by = 'Hour', ascending = True)

summer_2018_df.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Date,Month,Year,Hour
1524936,1252,2018-08-25 00:14:39.9810,2018-08-25 00:35:32.3120,317.0,E 6 St & Avenue B,40.724537,-73.981854,3087.0,Metropolitan Ave & Meeker Ave,40.714133,-73.952344,26859,Subscriber,1989,2,2018-08-25 00:14:39.981,8,2018,0
1524810,665,2018-08-25 00:07:49.1630,2018-08-25 00:18:54.8390,2002.0,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,3094.0,Graham Ave & Withers St,40.716981,-73.944859,25352,Subscriber,1988,2,2018-08-25 00:07:49.163,8,2018,0
1524809,178,2018-08-25 00:07:48.8770,2018-08-25 00:10:47.3300,438.0,St Marks Pl & 1 Ave,40.727791,-73.985649,433.0,E 13 St & Avenue A,40.729554,-73.980572,26414,Subscriber,1988,1,2018-08-25 00:07:48.877,8,2018,0
925377,581,2018-08-16 00:05:10.9400,2018-08-16 00:14:52.4530,516.0,E 47 St & 1 Ave,40.752069,-73.967844,167.0,E 39 St & 3 Ave,40.748901,-73.976049,15083,Subscriber,1963,1,2018-08-16 00:05:10.940,8,2018,0
1524898,197,2018-08-25 00:12:04.5880,2018-08-25 00:15:22.1390,3576.0,Park Pl & Vanderbilt Ave,40.676700,-73.969024,3580.0,St Johns Pl & Washington Ave,40.673724,-73.963161,16818,Subscriber,1988,2,2018-08-25 00:12:04.588,8,2018,0


In [4]:

num_allTrips_group = summer_2018_df.groupby('Month')

num_allTrips_df = pd.DataFrame(num_allTrips_group['bikeid'].count())

num_allTrips_df

,bikeid
Month,
6,1950575
7,1911333
8,1974825


What are the peak hours in which bikes are used during summer months? 

In [5]:
hours_group = summer_2018_df.groupby(['Hour'])
hours_df = pd.DataFrame(hours_group['Hour'].count())
hours_df = hours_df.sort_values(by = 'Hour', ascending = False)
hours_df = hours_df.rename(columns = { 'Hour':'Num Rides'})

hours_df.head(5)


,Num Rides
Hour,
17,570764
18,558404
8,435953
16,411263
19,409270


How has the proportion of short-term customers and annual subscribers changed?

In [6]:
trips_usertype_group = summer_2018_df.groupby(['Month','usertype'])
user_df = pd.DataFrame(trips_usertype_group['usertype'].count())

user_df = user_df.rename(columns = { 'usertype':'Num Users'})

user_df


Num Users
Month usertype             
6     Customer       258275
      Subscriber    1692300
7     Customer       284838
      Subscriber    1626495
8     Customer       273665
      Subscriber    1701160

Today, what are the top 10 stations in the city for starting a journey? (Based on data, why do you hypothesize these are the top locations?)

In [28]:
station_start_group = summer_2018_df.groupby('start station name')
station_start_df = pd.DataFrame(station_start_group['starttime'].count())
station_start_df = station_start_df.sort_values(by = 'starttime', ascending = False)

station_start_df = station_start_df.reset_index()

station_start_df = station_start_df.merge(summer_2018_df[['start station name','start station latitude','start station longitude']],
                                         how='left', on='start station name')

station_start_df = station_start_df.drop_duplicates()

station_start_df.head(5)

,start station name,starttime,start station latitude,start station longitude
0,Pershing Square North,45333,40.751873,-73.977706
45333,West St & Chambers St,37661,40.717548,-74.013221
82994,E 17 St & Broadway,35259,40.737050,-73.990093
118253,12 Ave & W 40 St,33775,40.760875,-74.002777
152028,Broadway & E 22 St,32768,40.740343,-73.989551


In [29]:
station_start_top_df = station_start_df.iloc[0:10,:]

station_start_top_df

,start station name,starttime,start station latitude,start station longitude
0,Pershing Square North,45333,40.751873,-73.977706
45333,West St & Chambers St,37661,40.717548,-74.013221
82994,E 17 St & Broadway,35259,40.737050,-73.990093
118253,12 Ave & W 40 St,33775,40.760875,-74.002777
152028,Broadway & E 22 St,32768,40.740343,-73.989551
184796,W 21 St & 6 Ave,32743,40.741740,-73.994156
217539,Broadway & E 14 St,30026,40.734546,-73.990741
247565,Grand Army Plaza & Central Park S,29829,40.764397,-73.973715
277394,Central Park S & 6 Ave,29259,40.765909,-73.976342
306653,South End Ave & Liberty St,29117,40.711512,-74.015756


In [30]:
station_end_group = summer_2018_df.groupby('end station name')
station_end_df = pd.DataFrame(station_end_group['stoptime'].count())
station_end_df = station_end_df.sort_values(by = 'stoptime', ascending = False)


station_end_df = station_end_df.reset_index()

station_end_df = station_end_df.merge(summer_2018_df[['end station name','end station latitude','end station longitude']],
                                         how='left', on='end station name')

station_end_df = station_end_df.drop_duplicates()

station_end_df.head(5)

,end station name,stoptime,end station latitude,end station longitude
0,Pershing Square North,45295,40.751873,-73.977706
45295,West St & Chambers St,39728,40.717548,-74.013221
85023,E 17 St & Broadway,36095,40.737050,-73.990093
121118,12 Ave & W 40 St,34378,40.760875,-74.002777
155496,W 21 St & 6 Ave,33210,40.741740,-73.994156


Today, what are the top 10 stations in the city for ending a journey? (Based on data, why?)

In [31]:
station_end_top_df = station_end_df.iloc[0:10,:]

station_end_top_df

,end station name,stoptime,end station latitude,end station longitude
0,Pershing Square North,45295,40.751873,-73.977706
45295,West St & Chambers St,39728,40.717548,-74.013221
85023,E 17 St & Broadway,36095,40.737050,-73.990093
121118,12 Ave & W 40 St,34378,40.760875,-74.002777
155496,W 21 St & 6 Ave,33210,40.741740,-73.994156
188706,Broadway & E 22 St,32796,40.740343,-73.989551
221502,Broadway & E 14 St,29636,40.734546,-73.990741
251138,Central Park S & 6 Ave,29317,40.765909,-73.976342
280455,W 20 St & 11 Ave,29252,40.746745,-74.007756
309707,South End Ave & Liberty St,29220,40.711512,-74.015756


Today, what are the bottom 10 stations in the city for starting a journey? (Based on data, why?)

In [32]:
station_start_group = summer_2018_df.groupby('start station name')
station_start_bottom_df = pd.DataFrame(station_start_group['starttime'].count())
station_start_bottom_df = station_start_bottom_df.sort_values(by = 'starttime', ascending = True)

station_start_bottom_df = station_start_df.iloc[0:10,:]

station_start_bottom_df

,start station name,starttime,start station latitude,start station longitude
0,Pershing Square North,45333,40.751873,-73.977706
45333,West St & Chambers St,37661,40.717548,-74.013221
82994,E 17 St & Broadway,35259,40.737050,-73.990093
118253,12 Ave & W 40 St,33775,40.760875,-74.002777
152028,Broadway & E 22 St,32768,40.740343,-73.989551
184796,W 21 St & 6 Ave,32743,40.741740,-73.994156
217539,Broadway & E 14 St,30026,40.734546,-73.990741
247565,Grand Army Plaza & Central Park S,29829,40.764397,-73.973715
277394,Central Park S & 6 Ave,29259,40.765909,-73.976342
306653,South End Ave & Liberty St,29117,40.711512,-74.015756


Today, what are the bottom 10 stations in the city for ending a journey (Based on data, why?)

In [33]:
station_end_group = summer_2018_df.groupby('end station name')
station_end_bottom_df = pd.DataFrame(station_end_group['stoptime'].count())
station_end_bottom_df = station_end_bottom_df.sort_values(by = 'stoptime', ascending = True)

station_end_bottom_df =  station_end_df.iloc[0:10,:]

station_end_bottom_df

,end station name,stoptime,end station latitude,end station longitude
0,Pershing Square North,45295,40.751873,-73.977706
45295,West St & Chambers St,39728,40.717548,-74.013221
85023,E 17 St & Broadway,36095,40.737050,-73.990093
121118,12 Ave & W 40 St,34378,40.760875,-74.002777
155496,W 21 St & 6 Ave,33210,40.741740,-73.994156
188706,Broadway & E 22 St,32796,40.740343,-73.989551
221502,Broadway & E 14 St,29636,40.734546,-73.990741
251138,Central Park S & 6 Ave,29317,40.765909,-73.976342
280455,W 20 St & 11 Ave,29252,40.746745,-74.007756
309707,South End Ave & Liberty St,29220,40.711512,-74.015756


Today, what is the gender breakdown of active participants (Male v. Female)?

In [34]:
gender_group = summer_2018_df.groupby(['gender'])
gender_df = pd.DataFrame(gender_group['Year'].count())
gender_df = gender_df.rename(columns = { 'Year':'Total'})
gender_df = gender_df.reset_index()
gender_df['Gender Type'] = np.where(gender_df['gender']==0, 'unknown', np.where(gender_df['gender']==1, 'male','female'))

gender_df

,gender,Total,Gender Type
0,0,617148,unknown
1,1,3815932,male
2,2,1403653,female


How does the average trip duration change by age?

In [35]:
tripDuration_group = summer_2018_df.groupby('birth year')
tripDuration_df = pd.DataFrame(tripDuration_group['tripduration'].sum())
tripDuration_df = tripDuration_df.reset_index()

numTrips_group = summer_2018_df.groupby('birth year')
numTrips_df = pd.DataFrame(numTrips_group['starttime'].count())
numTrips_df = numTrips_df.reset_index()


In [36]:
age_df = pd.merge(tripDuration_df,numTrips_df,how='left',on='birth year')

age_df['age'] = 2018 - age_df['birth year']
age_df['Avg Duration'] = age_df['tripduration'] / age_df['starttime']

age_df.head(5)

,birth year,tripduration,starttime,age,Avg Duration
0,1940,1362487,1955,78,696.924297
1,1941,1710096,2110,77,810.472038
2,1942,2061636,2644,76,779.741301
3,1943,1704117,1884,75,904.520701
4,1944,2142857,2874,74,745.600905


What is the average distance in miles that a bike is ridden?

In [37]:
summer_2018_df['lat_chg'] = summer_2018_df['end station latitude'] - \
                        summer_2018_df['start station latitude']

summer_2018_df['Dist in Miles'] = summer_2018_df['lat_chg'] * 69.2

summer_2018_df['Dist in Miles'] = summer_2018_df['Dist in Miles'].abs()



In [38]:
bikeId_miles_group = summer_2018_df.groupby('bikeid')
bikeID_miles_df = pd.DataFrame(bikeId_miles_group['Dist in Miles'].sum())
bikeID_miles_df = bikeID_miles_df.reset_index()

bikeId_numTrips_group = summer_2018_df.groupby('bikeid')
bikeId_numTrips_df = pd.DataFrame(bikeId_numTrips_group['Dist in Miles'].count())
bikeId_numTrips_df = bikeId_numTrips_df.reset_index()
bikeId_numTrips_df = bikeId_numTrips_df.rename(columns={'Dist in Miles':'Num_Trips'})

bikeID_avgMiles_df = pd.merge(bikeID_miles_df,bikeId_numTrips_df,how='left',on='bikeid')

bikeID_avgMiles_df['Avg_Miles'] = bikeID_avgMiles_df['Dist in Miles'] / bikeID_avgMiles_df['Num_Trips']

bikeID_avgMiles_df.head(5)


,bikeid,Dist in Miles,Num_Trips,Avg_Miles
0,14529,457.347472,525,0.871138
1,14530,77.059767,135,0.570813
2,14531,36.106057,43,0.839676
3,14532,289.298094,344,0.840983
4,14533,128.962144,178,0.724506


Which bikes (by ID) are most likely due for repair or inspection in the timespan? 

In [39]:
bikeID_avgMiles_df = bikeID_avgMiles_df.sort_values(by = 'Dist in Miles', ascending = False)

bike_repair_df = bikeID_avgMiles_df.iloc[0:10,:]
bike_repair_df


,bikeid,Dist in Miles,Num_Trips,Avg_Miles
9402,30168,947.864842,1012,0.936625
9203,29939,944.880819,930,1.016001
9925,30754,928.239637,986,0.941420
10031,30868,924.294758,953,0.969879
11975,33162,914.689534,975,0.938143
11249,32375,912.161360,932,0.978714
9263,30003,904.036253,943,0.958681
9704,30512,892.125397,927,0.962379
11122,32243,890.173459,906,0.982531
12091,33286,888.854773,919,0.967198


By what percentage has total ridership grown? 

In [40]:

membership_2018_q2 = pd.read_csv('membership_data/2018_q2.csv')
membership_2018_q3 = pd.read_csv('membership_data/2018_q3.csv')

In [41]:
membership_2018_q2_June = membership_2018_q2.iloc[61:90,:]

membership_2018_q3_July_August = membership_2018_q3.iloc[0:61,:]

In [42]:
membership_summer_2018_df =  pd.concat([membership_2018_q2_June,membership_2018_q3_July_August])
membership_summer_2018_df['Date'] = pd.to_datetime(membership_summer_2018_df['Date'])
membership_summer_2018_df['Month'] = membership_summer_2018_df['Date'].dt.month
membership_summer_2018_df['Year'] = membership_summer_2018_df['Date'].dt.year

In [43]:
membership_summer_2018_df.head(5)

,Date,Trips over the past 24-hours (midnight to 11:59pm),Miles traveled today (midnight to 11:59 pm),Total Annual Members (All Time),24-Hour Passes Purchased (midnight to 11:59 pm),3-Day Passes Purchased (midnight to 11:59 pm),Month,Year
61,2018-06-01,62890,147780,269167,3049,250,6,2018
62,2018-06-02,60287,158528,269371,5634,307,6,2018
63,2018-06-03,50041,134416,269544,3892,154,6,2018
64,2018-06-04,58910,130774,269763,2372,166,6,2018
65,2018-06-05,71906,155007,269934,2682,199,6,2018


In [44]:
membership_summer_2018_df.to_excel('membership_summer_2018.xlsx')

In [45]:
beg_June_2018_members = membership_summer_2018_df.iloc[0,:]['Total Annual Members (All Time)']
end_June_2018_members = membership_summer_2018_df.iloc[28,:]['Total Annual Members (All Time)']

beg_July_2018_members = membership_summer_2018_df.iloc[29,:]['Total Annual Members (All Time)']
end_July_2018_members = membership_summer_2018_df.iloc[59,:]['Total Annual Members (All Time)']

beg_August_2018_members = membership_summer_2018_df.iloc[60,:]['Total Annual Members (All Time)']
end_August_2018_members = membership_summer_2018_df.iloc[89,:]['Total Annual Members (All Time)']

In [46]:
print(beg_August_2018_members)
print(end_August_2018_members)

279888
284938


In [47]:

membership_totals_dict = {'beg_June':beg_June_2018_members,
                          'end_June':end_June_2018_members,
                          'beg_July':beg_July_2018_members,
                          'end_July':end_July_2018_members,
                          'beg_August':beg_August_2018_members,
                          'end_August':end_August_2018_members
                         }


membership_totals_dict

membership_totals = pd.DataFrame.from_dict(membership_totals_dict,orient='index')
membership_totals = membership_totals.reset_index()
membership_totals = membership_totals.rename(columns={'index': 'period',0:'Total'})

membership_totals['Per Change'] = (membership_totals['Total'] - membership_totals['Total'].shift(1)) / membership_totals['Total']*100

membership_totals

,period,Total,Per Change
0,beg_June,269167,NaN
1,end_June,274467,1.931015
2,beg_July,274813,0.125904
3,end_July,279702,1.747932
4,beg_August,279888,0.066455
5,end_August,284938,1.772315


In [48]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('citi_bike.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
num_allTrips_df.to_excel(writer,sheet_name = 'Total Trips')
hours_df.to_excel(writer,sheet_name = 'Peak Hours')
user_df.to_excel(writer,sheet_name = 'User Breakdown')

station_start_df.to_excel(writer,sheet_name = 'Station Start Mapping')
station_start_top_df.to_excel(writer,sheet_name = 'Station Start Top 10')
station_start_bottom_df.to_excel(writer,sheet_name = 'Station Start Bottom 10')

station_end_df.to_excel(writer,sheet_name = 'Station End Mapping')
station_end_top_df.to_excel(writer,sheet_name = 'Station End Top 10')
station_end_bottom_df.to_excel(writer,sheet_name = 'Station End Bottom 10')

gender_df.to_excel(writer,sheet_name = 'Gender Breakdown')
age_df.to_excel(writer,sheet_name = 'Age Breakdown')
bikeID_avgMiles_df.to_excel(writer,sheet_name = 'Avg Miles')
bike_repair_df.to_excel(writer,sheet_name = 'Bike Repair')
membership_totals.to_excel(writer,sheet_name = 'Ridershi Changes')

# Close the Pandas Excel writer and output the Excel file.
writer.save()